<a href="https://colab.research.google.com/github/habilg/keras_advanced/blob/master/keras_for_researchers_08_CustumTrainingLoop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Steps:
> prepare Data by data api. Dont forget shuffle, batch and...  
> select a loss  
> select an optimizer  
> put every thing in Gradient Tape
> inside gradianttape,
+ feed the batch to model
+ calculate the loss   
> claculate the calculus to the model trainable weights  
> update the model weights by optimizers

>
```python
for epoch in range(epochs):
  for batch,data in eumerate(dataset)
    with GradientTape as g:
      y_pred=model(data.x_train, Training=True)
      loss_value=loss_fn(data.y_train, y_pred)
      metrics=metric.update_state(data.y_train, y_pred)
    grads=g.gredient(loss_value,model.trainable_weights)
    opt.apply_gradients(zip(grads,model.trainable_weights))
    train_mae_val=metric.result()
    train_mae_val.reset_states()


```
